In [125]:
def get_str_oracle (X):
    return "{} {} {} {} {} {} {} {} {} {}".format (X[0], X[1], X[2], X[3], X[4], X[5], X[6], X[7], X[8], X[9])
def get_val_oracle (x, i):
    args = get_str_oracle (x[i])
    res = !./Oracle.static $args
    sys.stderr.write ("i:{} res:{}\n".format (i, res[0]))
    try:
        return float (res[0])
    except ValueError as vr:
#         print vr, "on res ",res[0], "data", args
        return None
my_data = np.genfromtxt('X_public.csv', delimiter=',', skip_header=1, usecols=(1,2,3,4,5,6,7,8,9,10))

In [126]:
import numpy as np
import sys
import csv
from xgboost import XGBRegressor

In [2]:
my_data = np.genfromtxt('X_public.csv', delimiter=',', skip_header=1, usecols=(1,2,3,4,5,6,7,8,9,10))

In [127]:
ls

get_y.py       Oracle.static*  Untitled.ipynb                y_0_999.csv
initial_x.txt  run.sh          validation.sample.submission  y_1000_1999.csv
learned.csv    submission.csv  X_public.csv                  y.txt


In [137]:
learned_indx =  np.genfromtxt ("learned.csv",delimiter=',')

In [231]:
initial_x = np.genfromtxt ("initial_x.txt")
y = np.genfromtxt ("y.txt")
learned_indx = np.genfromtxt ("learned.csv",delimiter=',')
learned_indx = set (learned_indx)

In [129]:
def get_str_oracle (X):
    return "{} {} {} {} {} {} {} {} {} {}".format (X[0], X[1], X[2], X[3], X[4], X[5], X[6], X[7], X[8], X[9])

In [130]:
def get_val_oracle (x, i):
    args = get_str_oracle (x[i])
    res = !./Oracle.static $args
    sys.stderr.write ("i:{} res:{}\n".format (i, res[0]))
    try:
        return float (res[0])
    except ValueError as vr:
#         print vr, "on res ",res[0], "data", args
        return None

In [6]:
# y = []
# for i, x in enumerate (initial_x):
#     get_val (initial_x, i)

In [7]:
y = np.genfromtxt ("y_0_999.csv")

In [8]:
y_1 = np.genfromtxt ('y_1000_1999.csv')

In [9]:
y = np.concatenate ((y,y_1), axis=0)

In [64]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [11]:
learned_indx = set (xrange(2000))

In [312]:
def new_epoch (initial_x, my_data, y, learned_indx):
    regressor = GradientBoostingRegressor (verbose=True,n_estimators=300,max_depth=6)
    pos_y = np.copy (y)
    pos_y[y<1] = 1
    regressor.fit (initial_x, np.log(pos_y))
    res = np.exp(regressor.predict (initial_x))
    learner = RandomForestRegressor (n_estimators=50,n_jobs=8)
    dif = np.abs (y - res)
    print "dif norm:",np.linalg.norm (dif)
    learner.fit (initial_x, dif)
    res_learner = learner.predict (my_data)
    desired = np.random.choice(np.argsort(res_learner)[-100:],size=5,replace=False)
    print desired
    rand_sample = np.random.randint (2000, high=initial_x.size, size=1)
    ask_oracle = np.concatenate ((desired,rand_sample),axis=0)
    for q in ask_oracle:
        if not q in learned_indx:
            oracle_res = [get_val_oracle (my_data, q)]
            learned_indx.add (q)
            initial_x = np.vstack ((initial_x, my_data[q])) 
            y = np.concatenate ((y, np.array (oracle_res)))
    return initial_x, my_data, y, learned_indx, regressor

In [313]:
for epoch in xrange (300):
    initial_x, my_data, y, learned_indx,_ = new_epoch (initial_x, my_data, y, learned_indx)
    print epoch

      Iter       Train Loss   Remaining Time 
         1         615.7587           15.83s
         2         500.1835           14.71s
         3         406.2896           14.10s
         4         330.1507           13.67s
         5         268.2996           13.54s
         6         218.0414           13.51s
         7         177.2091           13.34s
         8         144.0576           13.14s
         9         117.1263           12.94s
        10          95.2480           12.88s
        20          12.2855           12.30s
        30           1.7896           12.00s
        40           0.3930           11.82s
        50           0.1848           11.31s
        60           0.1414           10.63s
        70           0.1244            9.99s
        80           0.1149            9.36s
        90           0.1086            8.77s
       100           0.1032            8.21s
       200           0.0707            3.75s
       300           0.0571            0.00s
dif norm:

i:570533 res:46903019961073178606995201445529945534896241901568.00000000
i:243643 res:37851204180462997495189478551266099586084945002496.00000000
i:745305 res:39372758049411140185050082482771440690232876335104.00000000
i:142708 res:100.82195781


0
      Iter       Train Loss   Remaining Time 
         1         616.5003           13.48s
         2         500.7841           13.00s
         3         406.7759           12.66s
         4         330.5448           12.43s
         5         268.5658           12.64s
         6         218.2749           12.78s
         7         177.4138           12.68s
         8         144.1856           12.53s
         9         117.2311           12.39s
        10          95.3162           12.42s
        20          12.2897           12.19s
        30           1.7885           11.62s
        40           0.3902           11.04s
        50           0.1815           11.05s
        60           0.1376           10.63s
        70           0.1221           10.08s
        80           0.1122            9.61s
        90           0.1028            9.14s
       100           0.0980            8.62s
       200           0.0736            3.82s
       300           0.0571            0.00s
dif nor

i:932847 res:38629449188399763667856167213901368799646634213376.00000000
i:477370 res:13442697869230380180035664219341559274639393292288.00000000
i:763536 res:48904990569515707295176167910725091050249967894528.00000000
i:127460 res:23614912999919990350195873667216089133290799235072.00000000
i:35743 res:116549622353.86154175


1
      Iter       Train Loss   Remaining Time 
         1         617.4502           13.73s
         2         501.5117           13.02s
         3         407.4102           12.89s
         4         331.0395           12.81s
         5         268.9746           13.05s
         6         218.6170           13.18s
         7         177.6309           13.20s
         8         144.4241           13.25s
         9         117.4283           13.24s
        10          95.4975           13.25s
        20          12.3231           12.54s
        30           1.7950           12.13s
        40           0.3947           11.79s
        50           0.1849           11.33s
        60           0.1413           10.86s
        70           0.1253           10.30s
        80           0.1165            9.75s
        90           0.1090            9.21s
       100           0.1024            8.69s
       200           0.0733            3.90s
       300           0.0593            0.00s
dif nor

...

i:788553 res:16613692139260911637834686362646975140860925050880.00000000
i:83554 res:44878059468368469804593748704856375296.00000000


297
      Iter       Train Loss   Remaining Time 
         1         645.9590           14.87s
         2         524.7526           14.57s
         3         426.2698           14.47s
         4         346.2980           14.38s
         5         281.3619           14.32s
         6         228.6153           14.26s
         7         185.7662           14.17s
         8         150.9741           14.05s
         9         122.6783           14.00s
        10          99.7404           13.94s
        20          12.8361           13.88s
        30           1.8540           13.36s
        40           0.3986           12.76s
        50           0.1833           12.11s
        60           0.1395           11.39s
        70           0.1221           10.76s
        80           0.1125           10.14s
        90           0.1055            9.51s
       100           0.1009            8.91s
       200           0.0773            3.91s
       300           0.0617            0.00s
dif n

i:293633 res:44083446603456926649696819718345995440738856337408.00000000
i:190637 res:2234116.28911089


298
      Iter       Train Loss   Remaining Time 
         1         646.1778           14.67s
         2         524.8613           14.34s
         3         426.3776           14.26s
         4         346.4012           14.26s
         5         281.4532           14.28s
         6         228.7011           14.15s
         7         185.8260           14.07s
         8         151.0091           14.00s
         9         122.8077           14.02s
        10          99.8284           13.96s
        20          12.8462           12.95s
        30           1.8621           12.41s
        40           0.4034           11.83s
        50           0.1879           11.31s
        60           0.1426           10.84s
        70           0.1255           10.29s
        80           0.1160            9.67s
        90           0.1078            9.12s
       100           0.1024            8.57s
       200           0.0742            3.83s
       300           0.0572            0.00s
dif n

i:154218 res:293207.86754637


In [323]:
np.savetxt("initial_x.txt",initial_x)

In [324]:
np.savetxt ("y.txt", y)

In [325]:
cw = csv.writer(open("learned.csv",'w'))

In [326]:
cw.writerow(list(learned_indx))

In [327]:
initial_x, my_data, y, learned_indx, regr = new_epoch (initial_x, my_data, y, learned_indx)

      Iter       Train Loss   Remaining Time 
         1         646.1762           14.18s
         2         524.8599           13.88s
         3         426.3764           13.71s
         4         346.4006           13.76s
         5         281.4528           13.73s
         6         228.6448           13.68s
         7         185.7764           13.54s
         8         151.0141           13.40s
         9         122.7875           13.39s
        10          99.8083           13.38s
        20          12.8511           12.89s
        30           1.8658           12.30s
        40           0.4062           11.73s
        50           0.1895           11.20s
        60           0.1440           11.52s
        70           0.1273           10.88s
        80           0.1174           10.24s
        90           0.1110            9.59s
       100           0.1067            8.95s
       200           0.0775            3.99s
       300           0.0610            0.00s
dif norm:

i:121640 res:24881157930354466508995032032669556563091763757056.00000000
i:31757 res:34.64507568


In [329]:
y

array([  2.27521415e+15,   8.74656192e+21,   3.73455844e+02, ...,
         6.32484772e+28,   2.48811579e+49,   3.46450757e+01])

In [328]:
submission = np.exp (regr.predict (my_data))

In [330]:
submission

array([  2.83999396e+15,   9.68987217e+21,   3.24782390e+02, ...,
         9.83261786e+14,   3.17222751e+06,   1.37857796e+01])

In [331]:
f= open("submission.csv","w+")
f.write ("Id,Target\n")

for i,s in enumerate(submission):
     f.write("%d,%f\n" % (i+1, s))

f.close() 

In [332]:
!head X_private.csv

Id,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10
1,8.40187740,3.94382930,7.83099222,7.98440027,9.11647415,1.97551370,3.35222745,7.68229580,2.77774715,5.53969955
2,0.16300572,2.42886758,1.37231576,8.04176712,1.56679094,4.00944376,1.29790437,1.08808804,9.98924446,2.18256903
3,5.26744938,7.69913864,4.00228596,8.91529465,2.83314729,3.52458358,8.07724571,9.19026470,0.69755280,9.49327087
4,2.38279963,9.70634079,9.02208042,8.50919819,2.66665745,5.39760351,3.75206971,7.60248709,5.12535381,6.67723751
5,6.87861395,1.65974164,4.40104532,8.80075169,8.29201126,3.30337143,2.28968167,8.93372440,3.50360179,6.86669874
6,4.82490635,2.15824962,9.50252342,9.20128250,1.47660017,8.81062126,6.41080618,4.31953430,6.19596481,2.81059408
7,1.03171182,1.26075339,4.95444059,7.60475206,9.84751606,9.35003948,6.84445000,3.83188343,7.49770880,3.68663549
8,0.74529797,9.50104046,0.52529258,5.21563339,1.76210654,2.40062380,7.97798014,7.32654381,6.56563616,9.67405128
9,5.733185

In [334]:
x_private = np.genfromtxt('X_private.csv', delimiter=',', skip_header=1, usecols=(1,2,3,4,5,6,7,8,9,10))

In [335]:
submission = np.exp (regr.predict (x_private))

In [336]:
f= open("submission.csv","w+")
f.write ("Id,Target\n")

for i,s in enumerate(submission):
     f.write("%d,%f\n" % (i+1, s))

f.close() 

In [337]:
!head submission.csv

Id,Target
1,61302620076747464.000000
2,12.730119
3,525081058981743296512.000000
4,2212598234112.729004
5,1352351.837491
6,640938.946482
7,28.909247
8,4064.657220
9,1563163811972971495424.000000
